In [1]:
import cadquery as cq
from cadquery import exporters
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
from matplotlib import cm
import base64
from openai import OpenAI
from pydantic import BaseModel
import os

In [2]:
#Set your openai key
os.environ["OPENAI_API_KEY"] = 'YOUR OPEN AI KEY'

In [3]:
def read_image(image_path):
    image_path = "data/example_1.png"
    image = cv2.imread(image_path)

    # displaying image
    plt.imshow(image)
    plt.show()
    return image

def pre_process(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # apply a blur using the median filter
    image = cv2.medianBlur(image, 5)
    return image

def find_circle_cutouts(image):
    # finds the circles in the grayscale image using the Hough transform
    circles = cv2.HoughCircles(image=image, method=cv2.HOUGH_GRADIENT, dp=0.9,minDist=80, param1=110, param2=39, maxRadius=100)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # make a copy of the original image
    cimg = img.copy()
    for co, i in enumerate(circles[0, :], start=1):
        i = i.astype(int)
        # draw the outer circle in green
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle in red
        cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    print("Number of circles detected:", co)
    # save the image, convert to BGR to save with proper colors
    # cv2.imwrite("coins_circles_detected.png", cimg)
    # show the image
    plt.imshow(cimg)
    plt.show()
    return circles

def detect_object_edge(image):
    # finds the circles in the grayscale image using the Hough transform
    circles = cv2.HoughCircles(image=image, method=cv2.HOUGH_GRADIENT, dp=0.9,minDist=200, param1=110, param2=39, minRadius=100)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # make a copy of the original image
    cimg = img.copy()
    for co, i in enumerate(circles[0, :], start=1):
        i = i.astype(int)
        # draw the outer circle in green
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle in red
        cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    print("Number of circles detected:", co)
    # save the image, convert to BGR to save with proper colors
    # cv2.imwrite("coins_circles_detected.png", cimg)
    # show the image
    plt.imshow(cimg)
    plt.show()
    return circles


In [ ]:
image_path = "data/example_1.png"
image = read_image(image_path)
image = pre_process(image)
cutouts = find_circle_cutouts(image=image)
object_edge = detect_object_edge(image=image)

In [5]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def read_annotiations(image_path):
    client = OpenAI()

    # Getting the base64 string
    base64_image = encode_image(image_path)

    class Flange(BaseModel):
        outer_diameter: str
        inner_diameter: str
        small_holes_diameter: str
        distance_of_small_holes: str
        angle: str


    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Extract information about the flange's dimensions from this 2D technical drawing. Remove any degree signs if present",
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
        response_format=Flange,
    )

    flange = response.choices[0].message.parsed
    return flange

In [6]:
flange = read_annotiations(image_path=image_path)

In [7]:
# generate flange
# 2d coordinates

def generate_positions(flange):
    #TO DO: check if shape is flange - donut test
    outer_diameter = int(flange.outer_diameter)
    inner_diameter = int(flange.inner_diameter)
    small_holes_diameter = int(flange.small_holes_diameter)
    distance_of_small_holes = int(flange.distance_of_small_holes)
    angle = int(flange.angle)
    number_of_holes = int(360/angle)
    coords_flange = {
        "outer_circle": {
            "center": (0, 0, 0),  # Center of the flange
            "radius": outer_diameter / 2,    # Outer diameter divided by 2
        },
        "inner_circle": {
            "center": (0, 0, 0),  # Same center
            "radius": inner_diameter / 2,     # Inner diameter divided by 2
        },
        "holes": [
            # Calculate the hole positions based on the pattern diameter and angles
            {
                "center": (
                    (distance_of_small_holes / 2) * math.cos(math.radians(angle + i * 360 / number_of_holes)),  # X
                    (distance_of_small_holes / 2) * math.sin(math.radians(angle + i * 360 / number_of_holes)),  # Y
                    0,  # Z (all holes are on the same plane)
                ),
                "radius": small_holes_diameter / 2,  # Hole diameter divided by 2
            }
            for i in range(number_of_holes)
        ],
    }
    return coords_flange

In [ ]:
coords_flange = generate_positions(flange)
print(coords_flange)

In [ ]:
flange_thickness = 9  # Thickness of the flange

# Function to create a cylinder in 3D (used for the flange body and holes)
def create_cylinder(center, radius, height, num_points=100):
    theta = np.linspace(0, 2 * np.pi, num_points)
    x = center[0] + radius * np.cos(theta)
    y = center[1] + radius * np.sin(theta)
    z = np.array([0, height])
    X, Z = np.meshgrid(x, z)
    Y, _ = np.meshgrid(y, z)
    return X, Y, Z

# Function to create the flange surface
def create_flange(ax):
    # Outer cylinder (flange body)
    outer = coords_flange["outer_circle"]
    X, Y, Z = create_cylinder(outer["center"], outer["radius"], flange_thickness)
    ax.plot_surface(X, Y, Z, color='skyblue', alpha=0.8)

    # Inner cylinder (inner cut-out)
    inner = coords_flange["inner_circle"]
    X, Y, Z = create_cylinder(inner["center"], inner["radius"], flange_thickness)
    ax.plot_surface(X, Y, Z, color='white', alpha=1)

    # Holes
    for hole in coords_flange["holes"]:
        X, Y, Z = create_cylinder(hole["center"], hole["radius"], flange_thickness)
        ax.plot_surface(X, Y, Z, color='white', alpha=1)

# Create the plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Create the flange
create_flange(ax)

# Customize the plot
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
ax.set_box_aspect([1, 1, 0.3])  # Aspect ratio

ax.view_init(elev=30, azim=45)  # 3D View

plt.show()


In [10]:
def create_step_file(coords_flange):
    # Create the flange base
    outer_radius = coords_flange["outer_circle"]["radius"]
    inner_radius = coords_flange["inner_circle"]["radius"]

    # Create a 2D sketch for the flange
    flange = (
        cq.Workplane("XY")
        .circle(outer_radius)  # Outer circle
        .circle(inner_radius)  # Inner hole
        .extrude(10)           # Extrude to make a 3D solid
    )

    # Add holes
    for hole in coords_flange["holes"]:
        center = hole["center"]
        radius = hole["radius"]
        flange = flange.faces(">Z").workplane().pushPoints([(center[0], center[1])]).hole(radius * 2)

    # Export the flange to a STEP file
    output_file = "flange_example_1.step"
    cq.exporters.export(flange, output_file)
    print(f"STEP file saved as {output_file}")


In [ ]:
create_step_file(coords_flange=coords_flange)